In [1]:
import uproot
from glob import glob
from coffea.nanoevents import NanoEventsFactory
#SWAN 
base_directory = "/eos/user/d/dgrove/datasets/"

files = sorted(glob(f"{base_directory}TSlepSlep/*.root"))


file = uproot.open(files[0])
print("file info:")
dict(file)

file info:


{'tag;1': <TObjString 'untagged' at 0x7f9eeebf8430>,
 'Events;1': <TTree 'Events' (815 branches) at 0x7f9eeebf27f0>,
 'LuminosityBlocks;1': <TTree 'LuminosityBlocks' (2 branches) at 0x7f9eeebf26a0>,
 'Runs;1': <TTree 'Runs' (1443 branches) at 0x7f9eeebf21c0>,
 'MetaData;1': <TTree 'MetaData' (1 branches) at 0x7f9eeebf2280>,
 'ParameterSets;1': <TTree 'ParameterSets' (1 branches) at 0x7f9e8d7e0490>}

In [2]:
events = NanoEventsFactory.from_root(files[0]).events()

/eos/user/d/dgrove/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:205: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/eos/user/d/dgrove/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/nanoaod.py:205: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


In [3]:
import awkward as ak
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import mplhep


mplhep.style.use(mplhep.style.CMS)

In [4]:
# run this again to make sure the figure size updates (a known bug)
mplhep.style.use(mplhep.style.CMS)

## Create our preselection mask (cuts) to be used for every data set:

In [5]:
'''preselection_mask = events[(events.Electron.pt >= 5) & 
                                      (np.abs(events.Electron.eta) < 2.4) &
                                      (events.Electron.sip3d < 8) &
                                      (np.abs(events.Electron.dxy) < 0.05) & 
                                      (np.abs(events.Electron.dz) < 0.1) &
                                      (events.Electron.miniPFRelIso_all < (20 + 300/events.Electron.pt))]
print(len(events))
print(len(preselection_mask))'''

'preselection_mask = events[(events.Electron.pt >= 5) & \n                                      (np.abs(events.Electron.eta) < 2.4) &\n                                      (events.Electron.sip3d < 8) &\n                                      (np.abs(events.Electron.dxy) < 0.05) & \n                                      (np.abs(events.Electron.dz) < 0.1) &\n                                      (events.Electron.miniPFRelIso_all < (20 + 300/events.Electron.pt))]\nprint(len(events))\nprint(len(preselection_mask))'

In [6]:
# Create a per-electron mask based on preselection criteria
electron_mask = (
                (events.Electron.pt >= 5) & 
                (np.abs(events.Electron.eta) < 2.4) & 
                (events.Electron.sip3d < 8) & 
                (np.abs(events.Electron.dxy) < 0.05) & 
                (np.abs(events.Electron.dz) < 0.1) & 
                (events.Electron.miniPFRelIso_all < (20 + 300/events.Electron.pt))
)

In [7]:
muon_mask = (
                (events.Muon.pt >= 3) & 
                (np.abs(events.Muon.eta) < 2.4) & 
                (events.Muon.sip3d < 8) & 
                (np.abs(events.Muon.dxy) < 0.05) & 
                (np.abs(events.Muon.dz) < 0.1) & 
                (events.Muon.miniPFRelIso_all < (20 + 300/events.Muon.pt))
)

In [8]:
import coffea
coffea.__version__

'0.7.23'

In [55]:
from coffea.analysis_tools import PackedSelection

selection = PackedSelection()

#selection.add("twoElectron", ak.num(events.Electron) == 2)
selection.add("e pt", ak.any(events.Electron.pt >= 5, axis=1))
selection.add("e eta", ak.any(np.abs(events.Electron.eta) < 2.4, axis=1))
selection.add("e SIP3D", ak.any(events.Electron.sip3d < 8, axis=1))
selection.add("e dxy", ak.any(np.abs(events.Electron.dxy) < 0.05, axis=1))
selection.add("e dz", ak.any(np.abs(events.Electron.dz) < 0.1, axis=1))
selection.add("e miniPFRelIso", ak.any(events.Electron.miniPFRelIso_all < (20 + 300/events.Electron.pt), axis =1))

selection.add("mu pt", ak.any(events.Muon.pt >= 3, axis=1))
selection.add("mu eta", ak.any(np.abs(events.Muon.eta) < 2.4, axis=1))
selection.add("mu SIP3D", ak.any(events.Muon.sip3d < 8, axis=1))
selection.add("mu dxy", ak.any(np.abs(events.Muon.dxy) < 0.05, axis=1))
selection.add("mu dz", ak.any(np.abs(events.Muon.dz) < 0.1, axis=1))
selection.add("mu miniPFRelIso", ak.any(events.Muon.miniPFRelIso_all < (20 + 300/events.Muon.pt), axis=1))

print(selection)
print(len(events))
selected_events = events[selection.all("e pt", "e eta", "e SIP3D", "e dxy", "e dz", "e miniPFRelIso") | selection.all("mu pt", "mu eta", "mu SIP3D", "mu dxy", "mu dz", "mu miniPFRelIso")]

print(len(selected_events))


1274945


ValueError: 'electron' is not in list

In [52]:
print(f"The total number of events: {len(events)}")


# Create event-wise preselection mask: check if any electron or any muon passes the criteria
electron_event_mask = ak.any(electron_mask, axis=1)
muon_event_mask = ak.any(muon_mask, axis=1)

# Combine the electron and muon masks at the event level (OR condition)
preselection_mask = electron_event_mask | muon_event_mask

# Apply the preselection mask to select the events
selected_events = events[preselection_mask]

# Apply the mask event-wise, selecting events where at least one electron passes the cuts
#preselection_mask = ak.any(electron_mask | muon_mask, axis=1)
#preselection_mask = ak.any(electron_mask, axis=1)
# Apply the preselection mask (with electron cuts loaded) to the events
#selected_events = events[preselection_mask]
print(f"Event count after first preselection cuts on electrons: {len(selected_events)}")
# Repeat masking but now for muons:
# Apply the mask event-wise, selecting events where at least one muon passes the cuts
#preselection_mask = ak.any(muon_mask, axis=1)

# Apply the preselection mask (with electron cuts loaded) to the events
#selected_events = events[preselection_mask]

# Print the number of events before and after the preselection

#print(f"Event count after second preselection cuts on muons: {len(selected_events)}")
#print("So any events from here on in 'selected events' is guarentee to have at least one electron or muon in each event that passes the cuts")


#Some of the above code is wrong (and comments), what happens if you apply one mask and then the other is you end up losing a ton of events
#from the first mask that would've passed the second mask, maybe you want this, idk
#in my case, however, I want to guarentee there is at least 1 lepton (e or µ) that passes their respective cuts in an event.
#if you apply both masks like I did, it made sure there was BOTH an electron and a µ that passed their cuts, I don't need that. 
#The packed selection method a cell above works far better since I used a logical or | in the selection. it cuts ~43,000 events,
#those events have neither a passing electron nor a muon. very cool

The total number of events: 1274945
Event count after first preselection cuts on electrons: 1231253
Event count after second preselection cuts on muons: 1231253
So any events from here on in 'selected events' is guarentee to have at least one electron or muon in each event that passes the cuts


In [53]:
print(f"The total number of events: {len(events)}")

# Create event-wise preselection mask: check if any electron or any muon passes the criteria
electron_event_mask = ak.any(electron_mask, axis=1)
muon_event_mask = ak.any(muon_mask, axis=1)

# Combine the electron and muon masks at the event level (OR condition)
preselection_mask = electron_event_mask | muon_event_mask

# Apply the preselection mask to select the events
selected_events = events[preselection_mask]

print(f"Event count after preselection cuts: {len(selected_events)}")


The total number of events: 1274945
Event count after first preselection cuts: 1231253


For reference, from the [pdg id table](https://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf): 

$\tilde{e}^-_{L}$ = 1000011

$\tilde{µ}^-_L$ = 1000013

$\tilde{χ}^0_1$ = 1000022

$\tilde{χ}^0_2$ = 1000023

$\tilde{χ}^+_1$ = 1000024

$\tilde{e}^-_{R}$ = 2000011

$\tilde{µ}^-_R$ = 2000013

In [11]:
events.GenPart.fields

['eta',
 'mass',
 'phi',
 'pt',
 'genPartIdxMother',
 'pdgId',
 'status',
 'statusFlags',
 'genPartIdxMotherG',
 'distinctParentIdxG',
 'childrenIdxG',
 'distinctChildrenIdxG',
 'distinctChildrenDeepIdxG']

In [12]:
#selectron_L_events = events.
genparts = events.GenPart
print(genparts.pdgId[-1])
print(ak.any(genparts.pdgId == 1000011, axis=1))
selectrons_L_events = events.GenPart[ak.any(genparts.pdgId == 1000011, axis=1)]
print(len(events.GenPart[ak.any(genparts.pdgId == 1000011, axis=1)]))
print(len(events))

[-1, 1, 1000011, -1000011, 1000011, -1000011, ... 1000022, 2, 21, -2, 11, -11]
[False, False, False, True, False, True, ... False, False, True, False, False, True]
446511
1274945


In [48]:
#selectron_L_events = events.
selected_genparts = selected_events.GenPart
#print(genparts.pdgId[-1])
print(ak.any(selected_genparts.pdgId == 1000011, axis=1))
selected_te_L_events = selected_events[ak.any(selected_genparts.pdgId == 1000011, axis=1)]
print("events with preselection cuts and left handed selectrons:")
print(len(selected_te_L_events))
print("out of all selected events:")
print(len(selected_events))

selected_te_R_events = selected_events[ak.any(selected_genparts.pdgId == 2000011, axis=1)]
print("events with preselection cuts and right handed selectrons:")
print(len(selected_te_L_events))
print("out of all selected events:")
print(len(selected_events))

selected_tmu_L_events = selected_events[ak.any(selected_genparts.pdgId == 1000013, axis=1)]
print("events with preselection cuts and left handed smuons:")
print(len(selected_te_L_events))
print("out of all selected events:")
print(len(selected_events))

selected_tmu_R_events = selected_events[ak.any(selected_genparts.pdgId == 2000011, axis=1)]
print("events with preselection cuts and right handed smuons:")
print(len(selected_te_L_events))
print("out of all selected events:")
print(len(selected_events))

print("total number of events in raw NanoAOD file:")
print(len(events))

[False, False, False, True, False, True, ... False, False, True, False, False, True]
events with preselection cuts and left handed selectrons:
426474
out of all selected events:
1231442
events with preselection cuts and right handed selectrons:
426474
out of all selected events:
1231442
events with preselection cuts and left handed smuons:
426474
out of all selected events:
1231442
events with preselection cuts and right handed smuons:
426474
out of all selected events:
1231442
total number of events in raw NanoAOD file:
1274945
